In [1]:
import pandas as pd
import numpy as np
import re
import os
from z3 import *
from VeriGB import get_gamma_R

pd.set_option('display.max_columns', 500)


In [2]:
housing_data = pd.read_csv('./Data/kc_house_data.csv')

no_price_cols = [i for i in housing_data.columns if i not in {'price', 'date', 'id'}]

X = housing_data[no_price_cols]
y = housing_data['price']
X.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


First determine what 160 sqft was in terms of z-score

In [11]:
sqft_cols = [i for i in housing_data.columns if 'sqft' in i]

std_dev_all = []
for col in sqft_cols:
    std = np.std(X[col])
    std_dev_all.append(160/std)
    print(f'{col} 160/std = {160/std:.04f}')

sqft_living 160/std = 0.1742
sqft_lot 160/std = 0.0039
sqft_above 160/std = 0.1932
sqft_basement 160/std = 0.3615
sqft_living15 160/std = 0.2334
sqft_lot15 160/std = 0.0059


In [12]:
np.mean(std_dev_all)

0.16202214571393356

### Experiment

In [4]:
import pickle
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

np.random.seed(0)

X = housing_data[no_price_cols]
y = housing_data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

n = 1000*60*10  # 10 minutes
set_option(timeout=n)  # n = milliseconds from https://github.com/Z3Prover/z3/issues/1386

# tree_depth_options = [3,5,8,10]
# estimator_options = [50, 100, 200, 300, 400, 500]
# more complex regressors were taking too long so I scaled back sizes for depth=10
tree_depth_options = [10]
estimator_options = [50, 100, 200]

for max_depth in tree_depth_options:
    for estimators in estimator_options:
        print(f'Testing for estimators={estimators}, max_depth={max_depth}')

        gbr = GradientBoostingRegressor(
            max_depth=max_depth,
            learning_rate=0.1,
            n_estimators=estimators  # number of trees
        )

        gbr.fit(X_train_scaled, y_train)
        
        y_pred = gbr.predict(X_test_scaled)
        this_score = r2_score(y_test, y_pred)
        print(f"Test r2 score: {this_score:.02f}")

        # continue  # uncomment if you want to just see model prediction scores
        
        sat_count = 0
        timeout_count = 0
        number_sampled = 200

        sampled_indexes = np.random.randint(low=0, high=len(X_test_scaled), size=number_sampled)

        verbose = False
        for i, idx in enumerate(sampled_indexes):
            if verbose:
                print(f'testing sample {i}, index: {idx}')
            X_sample = X_test_scaled[idx]

            # I wanted to confirm that every value starts with the same initial prediction and that is true
        #     print(gbr.loss_.get_init_raw_predictions(
        #                 np.array(X_sample).reshape(1,-1), 
        #                 gbr.init_).astype(np.float64))  

            model_expression, all_reals = get_gamma_R(gbr, X_input=X_sample, epsilon=0.1, delta=100000)

            # define all real values
            exec(",".join(list(all_reals))+" = Reals('"+" ".join(list(all_reals))+"')")


            model_expression = re.sub("\s+", # one or more repetition of whitespace
                   '', # replace with empty string (->remove)
                   model_expression
                  )
            gamma_ = eval(model_expression)

            s = Solver()
            s.add(gamma_)
            solver_result = s.check()
            if verbose:
                print(solver_result)
            if s.check() == sat:
                sat_count+=1
                out_value = s.model()[out]
                if verbose: 
                    print(float(out_value.numerator_as_long())/float(out_value.denominator_as_long()))
            if s.check() == unknown:
                timeout_count+=1
                if verbose:
                    print('timeout found')

            if ((i+1)%50) == 0:
                print(f'Tested {i+1} cases so far')

        results_dict = {
            'sat_rate':sat_count/number_sampled,
            'unsat_rate': (number_sampled-sat_count-timeout_count)/number_sampled,
            'timeout_rate': timeout_count/number_sampled,
            'r2_score': this_score
        }

        print(results_dict)
        
        outfile_name = f'StandardizationResults/results_estimators={estimators}_depth={max_depth}.pkl'
        # check if file exists if so ask user if they want to overwrite
        if os.path.exists(outfile_name):
            while True:
                user_input = input(f'{outfile_name} already exists would you like to overwrite (yes) or skip (no)> ')
                if user_input in {'yes', 'no'}:
                    break  # break while loop
            if user_input == 'yes':
                pass  # file will be overwritten
            elif user_input=='no':
                continue  # to next test
        
        with open(outfile_name, 'wb+') as file:
            pickle.dump(results_dict, file)

Testing for estimators=50, max_depth=10
Test r2 score: 0.89
Tested 50 cases so far
Tested 100 cases so far
Tested 150 cases so far
Tested 200 cases so far
{'sat_rate': 0.505, 'unsat_rate': 0.495, 'timeout_rate': 0.0, 'r2_score': 0.8899451036987076}
Testing for estimators=100, max_depth=10
Test r2 score: 0.89
Tested 50 cases so far
Tested 100 cases so far
Tested 150 cases so far
Tested 200 cases so far
{'sat_rate': 0.565, 'unsat_rate': 0.315, 'timeout_rate': 0.12, 'r2_score': 0.8930563941475049}
Testing for estimators=200, max_depth=10
Test r2 score: 0.89
Tested 50 cases so far
Tested 100 cases so far
Tested 150 cases so far
Tested 200 cases so far
{'sat_rate': 0.385, 'unsat_rate': 0.005, 'timeout_rate': 0.61, 'r2_score': 0.8920609923753526}


In [7]:
with open(f'StandardizationResults/results_estimators={estimators}_depth={max_depth}.pkl', 'rb+') as file:
    results_out = pickle.load(file)

In [8]:
results_out

{'sat_rate': 0.385,
 'unsat_rate': 0.005,
 'timeout_rate': 0.61,
 'r2_score': 0.8920609923753526}

In [107]:
# print(X_sample)
# print(tree.feature)
# print(tree.threshold)

In [14]:
tree.value[4][0,0]

-84004.33994787265

In [15]:
tree.children_left


array([ 1,  2,  3, -1, -1,  6, -1, -1,  9, 10, -1, -1, 13, -1, -1],
      dtype=int64)

In [16]:
tree.children_right


array([ 8,  5,  4, -1, -1,  7, -1, -1, 12, 11, -1, -1, 14, -1, -1],
      dtype=int64)

In [17]:
tree

In [53]:
import re
exec('res='+re.sub('\s', '','4+\n\t\t5'))
res

9